In [ ]:
import os
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from macss import utility_functions
from macss import plotting_functions
from macss import admixture_functions
from scipy.stats import sigmaclip
from astropy.stats import biweight_location, biweight_scale
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor

In [ ]:
HOME = os.environ['HOME']
pz_dir = f'{HOME}/macss'

In [ ]:
train = tables_io.read(f"{pz_dir}/data/dp1_matched_v4_train.hdf5")
test = tables_io.read(f"{pz_dir}/data/dp1_matched_v4_test.hdf5")

In [ ]:
train_features = utility_functions.get_band_values(train, '{band}_gaap1p0Mag', 'ugrizy')
train_targets = train['redshift']
test_features = utility_functions.get_band_values(test, '{band}_gaap1p0Mag', 'ugrizy')
test_targets = test['redshift']

In [ ]:
knr = KNeighborsRegressor()

In [ ]:
def run_it(reg):
    preds = utility_functions.run_regression(reg, train_features, train_targets, test_features)
    _ = plotting_functions.plot_true_predict_fancy(test_targets, np.nan_to_num(preds, -0.4))
    _ = plotting_functions.plot_biweight_stats_v_redshift(test_targets, preds)

In [ ]:
run_it(knr)

In [ ]:
noise_grid = np.logspace(-4, 0, 17)

In [ ]:
def doit(reg, nclip=3):
    the_dict = {}
    inputs = np.hstack([train_features])
    est_orig = reg.predict(inputs)
    means = []
    stds = []
    outlier_fracs = []
    biweight_means = []
    biweight_stds = []
    clipped_fracs = []
    
    for noise in noise_grid:
        noisy_mags = admixture_functions.gaussian_noise(train_features, noise_levels=noise*np.ones(6)).T
        inputs = np.hstack([noisy_mags])
        ad_vals = reg.predict(inputs)
        deltas = (ad_vals - est_orig)/(1 + est_orig)
        subset_clip, _, _ = sigmaclip(deltas, low=3, high=3)
        for _j in range(nclip):
            subset_clip, _, _ = sigmaclip(subset_clip, low=3, high=3)

        the_dict[noise] = deltas
        clipped_fracs.append((len(deltas) - len(subset_clip))/(len(deltas)))
        outliers = (np.fabs(deltas) > 0.15).sum() / float(deltas.size)
        outlier_fracs.append(outliers)
        means.append(deltas.mean())
        stds.append(deltas.std())
        biweight_means.append(biweight_location(subset_clip))
        biweight_stds.append(biweight_scale(subset_clip))
    
    _ = plt.plot(noise_grid, means, label=r"mean $\delta z$", ls='-.', color='blue')
    _ = plt.plot(noise_grid, stds, label=r"RMS $\delta z$", ls='-.', color='green')
    _ = plt.plot(noise_grid, biweight_means, label=r"biweight mean $\delta z$", ls='-', color='blue')
    _ = plt.plot(noise_grid, biweight_stds, label=r"biweight RMS $\delta z$", ls='-', color='green')    
    _ = plt.plot(noise_grid, outlier_fracs, label=r"f $\frac{\delta z}{1+z} > 0.15$", ls='-', color='cyan')
#    _ = plt.plot(admix_grid, clipped_fracs, label=r"$f_{\rm outlier}$", ls='-.', color='cyan')
    _ = plt.xscale('log')
    _ = plt.legend()
    _ = plt.xlabel("Noise [mag]")
    _ = plt.ylabel(r"$\delta z$")

In [ ]:
doit(knr)